In [2]:
class State:
    def __init__(self, i, j, grid, cost):
        self.grid = grid
        self.i = i
        self.j = j
        self.n = len(grid)
        self.g = cost  

    def goalTest(self):
        return self.i == self.n - 1 and self.j == self.n - 1

    def moveGen(self):
        children = []
        dr = [-1, 1, 0, 0, -1, 1, -1, 1]
        dc = [0, 0, -1, 1, 1, -1, -1, 1]
        for k in range(8):
            nr = self.i + dr[k]
            nc = self.j + dc[k]
            if 0 <= nr < self.n and 0 <= nc < self.n and self.grid[nr][nc] == 0:
                children.append(State(nr, nc, self.grid, self.g + 1))
        return children

    def stepCost(self):
        return 1
    #manhattan distance
    def h(self):
        dx = abs((self.n - 1) - self.i)
        dy = abs((self.n - 1) - self.j)
        return dx + dy


In [3]:
def reconstructPath(pmap, node, g, f):
    path = [node]
    parent = pmap[node]
    while parent:
        path.append(parent)
        parent = pmap[parent]

    path = path[::-1]  
    print("Reconstructed Path with g, h, f values:")
    
    for step in path:
        print(f"({step.i}, {step.j}) -> g = {g[step]}, h = {step.h()}, f = {f[step]}")
    print("GOAL")

    return path



def propagateImprovement(node, g, f, pmap, CLOSED):
    for M in node.moveGen():
        k = M.stepCost()
        newg = k + g[node]
        if newg < g.get(M, float('inf')):
            pmap[M] = node
            g[M] = newg
            f[M] = g[M] + M.h()
            if M in CLOSED:
                propagateImprovement(M, g, f, pmap, CLOSED)


def A_Star(start):
    if start.grid[start.i][start.j] == 1:
        print("-1 (no path exists)")
        return []
    OPEN = [start]
    CLOSED = []
    pmap = {start: None}
    g = {start: 0}
    f = {start: g[start] + start.h()}

    while OPEN:
        node = min(OPEN, key=lambda x: f.get(x, float('inf')))
        OPEN.remove(node)
        if node.goalTest():
            return reconstructPath(pmap, node, g, f)
        else:
            CLOSED.append(node)
            for M in node.moveGen():
                k = M.stepCost()
                newg = k + g[node]
                if newg < g.get(M, float('inf')):
                    pmap[M] = node
                    g[M] = newg
                    f[M] = g[M] + M.h()
                    if M in CLOSED:
                        propagateImprovement(M, g, f, pmap, CLOSED)
                    elif M not in OPEN:
                        OPEN.append(M)
    return []


In [4]:
grid = [
    [0, 0, 0],
    [1, 1, 0],
    [1, 1, 0]
]
start = State(0, 0, grid, 0)
path = A_Star(start)


Reconstructed Path with g, h, f values:
(0, 0) -> g = 0, h = 4, f = 4
(0, 1) -> g = 1, h = 3, f = 4
(1, 2) -> g = 2, h = 1, f = 3
(2, 2) -> g = 3, h = 0, f = 3
GOAL


In [5]:
grid = [
    [1, 0, 0],
    [1, 1, 0],
    [1, 1, 0]
]
start = State(0, 0, grid, 0)
path = A_Star(start)

-1 (no path exists)


When comparing the two algorithms, Best First Search (Greedy Search) often finds a path more quickly because it relies solely on the heuristic (such as Manhattan or Euclidean distance) to guide its exploration. This makes it faster in terms of performance, as it expands fewer nodes on average. However, its drawback is that it does not guarantee the shortest path. In some cases, the greedy approach may overlook alternative routes that are slightly longer in heuristic distance but ultimately yield a shorter overall path length.

On the other hand, A* search balances both the cost of reaching a node (g-cost) and the estimated cost to the goal (h-cost). This ensures that when a path exists, A* always returns the shortest one, making it more reliable and optimal compared to Best First Search. The trade-off, however, is that A* usually explores more nodes than Greedy Search, which can increase computational overhead. In practice, A* provides accuracy and guarantees optimality, while Best First Search offers speed but at the cost of possible suboptimal results.
